In [71]:
#Import the modules/packages

import os
import pandas as pd
from pandas import ExcelWriter
from openpyxl import load_workbook
import urllib
import datetime
import time

#***"ATTN: Please seletct the first three letter of the month of interest for monthly data assembly:"***
active_month = "Apr"

#url for seasonally adjusted data:

esd_sa_url = "https://fortress.wa.gov/esd/employmentdata/docs/economic-reports/wa-historical-employment-seasonally-adjusted.xlsx"

#define the excel file name:

sa_filename = esd_sa_url[-49:-5]+"_"+time.strftime("%m%d%Y")+esd_sa_url[-5:]

In [73]:
time.strftime("%m%d%Y")

'05242016'

In [74]:
start = time.time()
print "hello Jalal Jahir what"
print "hello Jalal Jahir is a GREAT Guy"
end = time.time()
processtime = end - start
print "{0: .3}".format(processtime)
print float(processtime)

hello Jalal Jahir what
hello Jalal Jahir is a GREAT Guy
 0.0
0.0


In [75]:
print os.getcwd()

J:\Staff\Jalal\Monthly_Emp_Update_Scratch\wa_esd_emp


In [76]:
current_dir = os.getcwd()

In [126]:
#download the data
def download_esd_sa():
    start = time.time()
    esdfile = urllib.URLopener()
    esdfile.retrieve(esd_sa_url, os.getcwd()+"/"+ sa_filename)
    end = time.time()
    processtime = end - start
    print "Data download completed. Download Time: " + "{0: .3}".format(processtime) + " Sec."
    

SyntaxError: invalid syntax (<ipython-input-126-c2baec72ce2a>, line 5)

In [110]:
download_esd_sa()

Data download completed. Download Time:  1.12 Sec.


In [13]:
help (urllib)

Help on module urllib:

NAME
    urllib - Open an arbitrary URL.

FILE
    c:\users\jalal\anaconda2\lib\urllib.py

DESCRIPTION
    See the following document for more info on URLs:
    "Names and Addresses, URIs, URLs, URNs, URCs", at
    http://www.w3.org/pub/WWW/Addressing/Overview.html
    
    See also the HTTP spec (from which the error codes are derived):
    "HTTP - Hypertext Transfer Protocol", at
    http://www.w3.org/pub/WWW/Protocols/
    
    Related standards and specs:
    - RFC1808: the "relative URL" spec. (authoritative status)
    - RFC1738 - the "URL standard". (authoritative status)
    - RFC1630 - the "URI spec". (informational status)
    
    The object returned by URLopener().open(file) will differ per
    protocol.  All you know is that is has methods read(), readline(),
    readlines(), fileno(), close() and info().  The read*(), fileno()
    and close() methods work like those of open files.
    The info() method returns a mimetools.Message object which can b

In [111]:
# Check the number of worksheet in the .xlsx file & list the name of the worksheets
#included in the analysis

def check_worksheet():
    esd_sa_file = pd.ExcelFile(os.getcwd()+"/"+ sa_filename)
    #get the list of worksheets from the excelfile
    esd_sa_ws_master = {sheet_name: esd_sa_file.parse(sheet_name) 
              for sheet_name in esd_sa_file.sheet_names}

    a_list = []
    count = 0
    count_total = 0
    for i in esd_sa_ws_master:
        if i == "Seattle MSA":
            a_list.append(i)
            count = count+1
            count_total = count_total+1
        elif i == "Bremerton MSA":
            a_list.append(i)
            count = count+1
            count_total = count_total+1
        elif i == "Tacoma MSA":
            a_list.append(i)
            count = count+1
            count_total = count_total+1
        else:
            count_total = count_total+1


    print "List of worksheets to be included in the analysis: "

    count_ws = 0
    for x in sorted(a_list):
        count_ws = count_ws+1 
        print str(count_ws)+". "+ x

    print "Total "+str(count)+" "+"out of "+ str(count_total)+ " worksheets from the excel file include data for the PSRC region."

check_worksheet()

List of worksheets to be included in the analysis: 
1. Bremerton MSA
2. Seattle MSA
3. Tacoma MSA
Total 3 out of 12 worksheets from the excel file include data for the PSRC region.


In [94]:
## EXAMPLE: Read the worksheet into dataframe from a excel file:

#esd_sa_file = pd.ExcelFile(os.getcwd()+"/"+ sa_filename)
#df_sa_kin_sno = esd_sa_file.parse("Seattle MSA", header =1).head()

In [5]:
#read the required worksheets into dataframes

df_sa_kin_sno = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Seattle MSA", header = 1, index_col=[1])
df_sa_kit = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Bremerton MSA", header = 1, index_col=[1])
df_sa_pie = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Tacoma MSA", header = 1, index_col=[1])


In [183]:
type(df_sa_pie)

pandas.core.frame.DataFrame

In [182]:
df_sa_pie.shape

(315, 11)

In [185]:
df_sa_pie.dtypes

id
year                      int64
month                    object
Pierce: Total NonFarm     int64
Pierce: Consruction       int64
Pierce: FIRE              int64
Pierce: Manufacturing     int64
Pierce: Retail            int64
Pierce: Service           int64
Pierce: WTU               int64
Pierce: Government        int64
Pierce: Education         int64
dtype: object

In [188]:
df_sa_pie.columns.tolist()

['year',
 'month',
 'Pierce: Total NonFarm',
 'Pierce: Consruction',
 'Pierce: FIRE',
 'Pierce: Manufacturing',
 'Pierce: Retail',
 'Pierce: Service',
 'Pierce: WTU',
 'Pierce: Government',
 'Pierce: Education']

In [119]:
df_sa_kin_sno_main

[]

In [175]:
#initial process of King and Snohomish County data:

def sa_kin_sno():
    #read the required worksheets into dataframes
    df_sa_kin_sno = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Seattle MSA", header = 1, index_col=[1])

    #transpose the dataframe
    df_sa_kin_sno = df_sa_kin_sno.transpose()

    #transposing caused the date column to be the index column, so
    #extracting the date information to another column named "date_stamp"

    df_sa_kin_sno['date_stamp'] = df_sa_kin_sno.index

    #deleting the first data row (not the header) containing NAICS info 
    #entirely for formatting purposes 

    df_sa_kin_sno = df_sa_kin_sno.ix[1:]

    #resetting the index of the dataframe
    #as we already copied the date info into another column, nothing to worry

    df_sa_kin_sno = df_sa_kin_sno.reset_index(drop =True)

    #renaming the index colum from "NAICS Industy" to "id" 

    df_sa_kin_sno.columns.names = ["id"]

    #get a column for year and one for month (note the foramting for month)
    df_sa_kin_sno["year"] = df_sa_kin_sno["date_stamp"].dt.year
    df_sa_kin_sno["month"] = df_sa_kin_sno["date_stamp"].dt.strftime("%b")

    #get the month column to string (not essential)
    df_sa_kin_sno["month"] = df_sa_kin_sno["month"].astype("str")


#define sectors for King and Snohomish Counties:


    #get the list of column names, any of the options below works just fine
    # list(df_sa_kin_sno) or list(df_sa_kin_sno.columns.values) works too
    kin_sno_list = df_sa_kin_sno.columns.values.tolist()


    #define PSRC sectors by adding columns for each of the eight sectors and total nonfarm jobs:
    df_sa_kin_sno["King & Snohomish: Total NonFarm"] = df_sa_kin_sno["Total Nonfarm"] 
    df_sa_kin_sno["King & Snohomish: Consruction"] = df_sa_kin_sno[kin_sno_list[3]] + df_sa_kin_sno[kin_sno_list[4]]
    df_sa_kin_sno["King & Snohomish: FIRE"] = df_sa_kin_sno[kin_sno_list[33]]
    df_sa_kin_sno["King & Snohomish: Manufacturing"] = df_sa_kin_sno[kin_sno_list[8]]
    df_sa_kin_sno["King & Snohomish: Retail"] = df_sa_kin_sno[kin_sno_list[20]]
    df_sa_kin_sno["King & Snohomish: Service"] = df_sa_kin_sno[kin_sno_list[17]] - df_sa_kin_sno[kin_sno_list[18]] - df_sa_kin_sno[kin_sno_list[33]] - df_sa_kin_sno[kin_sno_list[62]]                                                     
    df_sa_kin_sno["King & Snohomish: WTU"] = df_sa_kin_sno[kin_sno_list[18]] - df_sa_kin_sno[kin_sno_list[20]]
    df_sa_kin_sno["King & Snohomish: Government"] = df_sa_kin_sno[kin_sno_list[62]] - df_sa_kin_sno[kin_sno_list[65]] - df_sa_kin_sno[kin_sno_list[67]]
    df_sa_kin_sno["King & Snohomish: Education"] = df_sa_kin_sno[kin_sno_list[65]] + df_sa_kin_sno[kin_sno_list[67]]

    #delete columns that we don't need anymore (all except the year, month and columns created for sectors)
    #df_sa_kin_sno = df_sa_kin_sno.drop(df_sa_kin_sno.columns[range(69)], axis = 1) works for this operation too!
    df_sa_kin_sno = df_sa_kin_sno.drop(df_sa_kin_sno.columns[:69], axis = 1)

    #ATTENTION: """just for this iteration, won't need for future runs""", deleting the row that contains Apr16 info for Seattle MSA
    #df_sa_kin_sno = df_sa_kin_sno.drop(df_sa_kin_sno.index[[315]])

    #putting in a check-point in place to make sure the sector columns add up to the total jobs:
    #df_sa_kit["Kitsap: Total NonFarm"].isin(df_sa_kit[kit_col_list[3:]].sum(axis=1)).all() == True.... works too for this step.
    
    

    if df_sa_kin_sno["King & Snohomish: Total NonFarm"].isin(df_sa_kin_sno.ix[:,3:].sum(axis=1)).all() == True:
        print "Checked: King & Snohomish Counties - All industry sectors adds up to total NonFarm jobs..."
    else:
        print "ERROR!!"
    return df_sa_kin_sno


In [176]:
df_sa_kin_sno = sa_kin_sno()



Checked: King & Snohomish Counties - All industry sectors adds up to total NonFarm jobs...


In [177]:
df_sa_kin_sno.tail()

id,year,month,King & Snohomish: Total NonFarm,King & Snohomish: Consruction,King & Snohomish: FIRE,King & Snohomish: Manufacturing,King & Snohomish: Retail,King & Snohomish: Service,King & Snohomish: WTU,King & Snohomish: Government,King & Snohomish: Education
311,2015,Dec,1614000,89400,82900,170100,169700,759800,127200,110600,104300
312,2016,Jan,1619100,89800,84300,168900,173500,761900,127700,110400,102600
313,2016,Feb,1627000,91700,84000,167800,174300,766700,127500,110900,104100
314,2016,Mar,1629200,92500,84300,167600,173800,765400,129300,112000,104300
315,2016,Apr,1634100,91900,84700,167600,174900,769400,129100,112300,104200


In [132]:
def sa_kit():
    df_sa_kit = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Bremerton MSA", header = 1, index_col=[1])
    df_sa_kit = df_sa_kit.transpose()
    df_sa_kit['date_stamp'] = df_sa_kit.index
    df_sa_kit = df_sa_kit.ix[1:]
    df_sa_kit = df_sa_kit.reset_index(drop =True)
    df_sa_kit.columns.names = ["id"]
    df_sa_kit["year"] = df_sa_kit["date_stamp"].dt.year
    df_sa_kit["month"] = df_sa_kit["date_stamp"].dt.strftime("%b")

    #get the month column to string (not essential)
    df_sa_kit["month"] = df_sa_kit["month"].astype("str")

    kit_list = df_sa_kit.columns.values.tolist()

    df_sa_kit["Kitsap: Total NonFarm"] = df_sa_kit["Total Nonfarm"] 
    df_sa_kit["Kitsap: Consruction"] = df_sa_kit[kit_list[3]]
    #add new column for FIRE with zero as values with index aligned with the dataframe
    df_sa_kit["Kitsap: FIRE"] = pd.Series([0 for x in range(len(df_sa_kit.index))], index=df_sa_kit.index)
    df_sa_kit["Kitsap: Manufacturing"] = df_sa_kit[kit_list[4]]
    df_sa_kit["Kitsap: Retail"] = df_sa_kit[kit_list[8]]
    df_sa_kit["Kitsap: Service"] = df_sa_kit[kit_list[5]] - df_sa_kit[kit_list[7]] - df_sa_kit[kit_list[11]]
    df_sa_kit["Kitsap: WTU"] = df_sa_kit[kit_list[7]] - df_sa_kit[kit_list[8]]
    df_sa_kit["Kitsap: Government"] = df_sa_kit[kit_list[11]]
    df_sa_kit["Kitsap: Education"] = pd.Series([0 for x in range(len(df_sa_kit.index))], index=df_sa_kit.index)

    df_sa_kit = df_sa_kit.drop(df_sa_kit.columns[:16], axis = 1)

    if df_sa_kit["Kitsap: Total NonFarm"].isin(df_sa_kit.ix[:,3:].sum(axis=1)).all() == True:
        print "Checked: Kitsap County - All industry sectors adds up to total NonFarm jobs..."
    else:
        print "ERROR!!"
    return pd.DataFrame[df_sa_kit]




In [133]:
sa_kit().tail()

Checked: Kitsap County - All industry sectors adds up to total NonFarm jobs...


id,year,month,Kitsap: Total NonFarm,Kitsap: Consruction,Kitsap: FIRE,Kitsap: Manufacturing,Kitsap: Retail,Kitsap: Service,Kitsap: WTU,Kitsap: Government,Kitsap: Education
311,2015,Dec,88800,4200,0,2400,10600,37100,2600,31900,0
312,2016,Jan,88700,4100,0,2500,10700,37100,2500,31800,0
313,2016,Feb,88700,4000,0,2500,11000,36800,2600,31800,0
314,2016,Mar,88800,4200,0,2500,11000,36700,2600,31800,0
315,2016,Apr,88600,4200,0,2500,11000,36600,2600,31700,0


In [134]:
def sa_pie():
    df_sa_pie = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Tacoma MSA", header = 1, index_col=[1])
    df_sa_pie = df_sa_pie.transpose()
    df_sa_pie['date_stamp'] = df_sa_pie.index
    df_sa_pie = df_sa_pie.ix[1:]
    df_sa_pie = df_sa_pie.reset_index(drop =True)
    df_sa_pie.columns.names = ["id"]
    df_sa_pie["year"] = df_sa_pie["date_stamp"].dt.year
    df_sa_pie["month"] = df_sa_pie["date_stamp"].dt.strftime("%b")

    #get the month column to string (not essential)
    df_sa_pie["month"] = df_sa_pie["month"].astype("str")

    pie_list = df_sa_pie.columns.values.tolist()

    df_sa_pie["Pierce: Total NonFarm"] = df_sa_pie["Total Nonfarm"] 
    df_sa_pie["Pierce: Consruction"] = df_sa_pie[pie_list[3]] + df_sa_pie[pie_list[4]]
    df_sa_pie["Pierce: FIRE"] = df_sa_pie[pie_list[15]]
    df_sa_pie["Pierce: Manufacturing"] = df_sa_pie[pie_list[6]]
    df_sa_pie["Pierce: Retail"] = df_sa_pie[pie_list[10]]
    df_sa_pie["Pierce: Service"] = df_sa_pie[pie_list[7]] - df_sa_pie[pie_list[8]] - df_sa_pie[pie_list[15]] - df_sa_pie[pie_list[25]]                                                     
    df_sa_pie["Pierce: WTU"] = df_sa_pie[pie_list[8]] - df_sa_pie[pie_list[10]]
    df_sa_pie["Pierce: Government"] = df_sa_pie[pie_list[25]] - df_sa_pie[pie_list[28]] - df_sa_pie[pie_list[30]]
    df_sa_pie["Pierce: Education"] = df_sa_pie[pie_list[28]] + df_sa_pie[pie_list[30]]

    df_sa_pie = df_sa_pie.drop(df_sa_pie.columns[:32], axis = 1)    

    if df_sa_pie["Pierce: Total NonFarm"].isin(df_sa_pie.ix[:,3:].sum(axis=1)).all() == True:
        print "Checked: Pierce County - All industry sectors adds up to total NonFarm jobs..."
    else:
        print "ERROR!!"
    return df_sa_pie




In [135]:
sa_pie().tail()

Checked: Pierce County - All industry sectors adds up to total NonFarm jobs...


id,year,month,Pierce: Total NonFarm,Pierce: Consruction,Pierce: FIRE,Pierce: Manufacturing,Pierce: Retail,Pierce: Service,Pierce: WTU,Pierce: Government,Pierce: Education
311,2015,Dec,300800,20400,13600,17100,36400,129300,26400,34900,22700
312,2016,Jan,301600,20500,13700,16900,37100,129400,26300,35100,22600
313,2016,Feb,304000,21200,13800,17100,37800,129700,26500,35200,22700
314,2016,Mar,305000,21600,13800,17000,38200,130000,26500,35300,22600
315,2016,Apr,304000,21200,13700,17000,38000,129600,26900,35100,22500


In [136]:
#merge the three dataframes to produce a regional dataframe
def sa_region():
    df_sa_psrc = pd.merge(pd.merge(df_sa_kin_sno, df_sa_pie, on = ["year", "month"]), df_sa_kit, on = ["year", "month"])
    
    #define regional sectors and add columns to a new dataframe

    df_sa_region  = df_sa_psrc
    region_list = df_sa_region.columns.values.tolist()

    df_sa_region["Region: Construction"] = df_sa_region[region_list[3]] + df_sa_region[region_list[12]] + df_sa_region[region_list[21]]
    df_sa_region["Region: FIRE"] = df_sa_region[region_list[4]] + df_sa_region[region_list[13]] + df_sa_region[region_list[22]]
    df_sa_region["Region: Manufacturing"] = df_sa_region[region_list[5]] + df_sa_region[region_list[14]] + df_sa_region[region_list[23]]
    df_sa_region["Region: Retail"] = df_sa_region[region_list[6]] + df_sa_region[region_list[15]] + df_sa_region[region_list[24]]
    df_sa_region["Region: Service"] = df_sa_region[region_list[7]] + df_sa_region[region_list[16]] + df_sa_region[region_list[25]]
    df_sa_region["Region: WTU"] = df_sa_region[region_list[8]] + df_sa_region[region_list[17]] + df_sa_region[region_list[26]]
    df_sa_region["Region: Education"] = df_sa_region[region_list[10]] + df_sa_region[region_list[19]] + df_sa_region[region_list[28]]
    df_sa_region["Region: Government"] = df_sa_region[region_list[9]] + df_sa_region[region_list[18]] + df_sa_region[region_list[27]]
    df_sa_region["Region: Total NonFarm"] = df_sa_region[region_list[2]] + df_sa_region[region_list[11]] + df_sa_region[region_list[20]]
    
    return df_sa_region

In [125]:
#merge the three dataframes to produce a regional dataframe
df_sa_psrc = pd.merge(pd.merge(df_sa_kin_sno, df_sa_pie, on = ["year", "month"]), df_sa_kit, on = ["year", "month"])

In [84]:
#define regional sectors and add columns to a new dataframe

df_sa_region  = df_sa_psrc
region_list = df_sa_region.columns.values.tolist()

df_sa_region["Region: Construction"] = df_sa_region[region_list[3]] + df_sa_region[region_list[12]] + df_sa_region[region_list[21]]
df_sa_region["Region: FIRE"] = df_sa_region[region_list[4]] + df_sa_region[region_list[13]] + df_sa_region[region_list[22]]
df_sa_region["Region: Manufacturing"] = df_sa_region[region_list[5]] + df_sa_region[region_list[14]] + df_sa_region[region_list[23]]
df_sa_region["Region: Retail"] = df_sa_region[region_list[6]] + df_sa_region[region_list[15]] + df_sa_region[region_list[24]]
df_sa_region["Region: Service"] = df_sa_region[region_list[7]] + df_sa_region[region_list[16]] + df_sa_region[region_list[25]]
df_sa_region["Region: WTU"] = df_sa_region[region_list[8]] + df_sa_region[region_list[17]] + df_sa_region[region_list[26]]
df_sa_region["Region: Education"] = df_sa_region[region_list[10]] + df_sa_region[region_list[19]] + df_sa_region[region_list[28]]
df_sa_region["Region: Government"] = df_sa_region[region_list[9]] + df_sa_region[region_list[18]] + df_sa_region[region_list[27]]
df_sa_region["Region: Total NonFarm"] = df_sa_region[region_list[2]] + df_sa_region[region_list[11]] + df_sa_region[region_list[20]]

In [140]:
def sa_region_query():
    #Query: just the three MSA and regional total
    df_sa_region_total = df_sa_region[[0,1, 2, 11,20,-1]]
    #Query: just the three MSA and regional total for current month
    df_sa_region_total_CM = df_sa_region[[0,1, 2, 11,20,-1]][df_sa_region.month== active_month]
    #Query: just the three MSA and regional sectors for current month
    df_sa_region_sector_CM = df_sa_region[df_sa_region.month== active_month]
    


In [143]:
sa_region_query.tail(3)

TypeError: sa_region_query() takes no arguments (1 given)

In [87]:
#delete unnecessary worksheets from the excel file

def delete_ws():
    wb_sa = load_workbook(os.getcwd()+"/"+ sa_filename)

    del_sheets = []

    for i in wb_sa.get_sheet_names():
        if i == "Seattle MSA":
            pass
        elif i == "Tacoma MSA":
            pass
        elif i == "Bremerton MSA":
            pass
        elif i == "Washington State":
            pass
        else:
            del_sheets.append(i)
    #print del_sheets
    #now delete the bad sheets
    [wb_sa.remove_sheet(wb_sa.get_sheet_by_name(sheet)) for sheet in del_sheets]
    #if you get a ValueError, this means you are incorrectly requesting a sheet

    wb_sa.save(os.getcwd()+"/"+ sa_filename)    

delete_ws()
    
    

In [88]:
#write the data frames to the excel file on different worksheets with different names:

def data_to_workbook():
    sa_book = load_workbook(os.getcwd()+"/"+ sa_filename)
    sa_writer = pd.ExcelWriter(os.getcwd()+"/"+ sa_filename, engine='openpyxl') 
    sa_writer.book = sa_book
    sa_writer.sheets = dict((ws.title, ws) for ws in sa_book.worksheets)

    df_sa_region_total.to_excel(sa_writer, "RegionMaster", index =False)

    df_sa_region.to_excel(sa_writer, "Region_Sector", index =False)

    df_sa_region_total_CM.to_excel(sa_writer, "Region_CMonth", index =False)

    df_sa_region_sector_CM.to_excel(sa_writer, "Region_Sec_CMonth", index =False)

    sa_writer.save()
    
    print "Data frames exported to the exce workbook located at the working directory:  "+"\n"+ os.getcwd()


Data frames exported to the exce workbook located at the working directory:  
J:\Staff\Jalal\Monthly_Emp_Update_Scratch\wa_esd_emp


In [ ]:
download_esd_sa()
check_worksheet()
sa_kin_sno()
sa_kit()
sa_pie()
sa_region()
delete_ws()

data_to_workbook()




In [147]:
print "\nJalal"


Jalal


In [47]:

#[df_sa_psrc.month== active_month]

In [51]:
df_sa_region_total_CM = df_sa_region[[0,1, 2, 11,20,-1]][df_sa_psrc.month== active_month]

In [66]:
df_sa_region_sector_CM = df_sa_region[df_sa_region.month== active_month]


In [14]:
#define regional sectors and add columns to a new dataframe


df_sa_psrc["Region: Construction"] = df_sa_psrc[psrc_list[3]] + df_sa_psrc[psrc_list[12]] + df_sa_psrc[psrc_list[21]]
df_sa_psrc["Region: FIRE"] = df_sa_psrc[psrc_list[4]] + df_sa_psrc[psrc_list[13]] + df_sa_psrc[psrc_list[22]]
df_sa_psrc["Region: Manufacturing"] = df_sa_psrc[psrc_list[5]] + df_sa_psrc[psrc_list[14]] + df_sa_psrc[psrc_list[23]]
df_sa_psrc["Region: Retail"] = df_sa_psrc[psrc_list[6]] + df_sa_psrc[psrc_list[15]] + df_sa_psrc[psrc_list[24]]
df_sa_psrc["Region: Service"] = df_sa_psrc[psrc_list[7]] + df_sa_psrc[psrc_list[16]] + df_sa_psrc[psrc_list[25]]
df_sa_psrc["Region: WTU"] = df_sa_psrc[psrc_list[8]] + df_sa_psrc[psrc_list[17]] + df_sa_psrc[psrc_list[26]]
df_sa_psrc["Region: Education"] = df_sa_psrc[psrc_list[10]] + df_sa_psrc[psrc_list[19]] + df_sa_psrc[psrc_list[28]]
df_sa_psrc["Region: Government"] = df_sa_psrc[psrc_list[9]] + df_sa_psrc[psrc_list[18]] + df_sa_psrc[psrc_list[27]]
df_sa_psrc["Region: Total NonFarm"] = df_sa_psrc[psrc_list[2]] + df_sa_psrc[psrc_list[11]] + df_sa_psrc[psrc_list[20]]

In [108]:
#(df_sa_psrc[(df_sa_psrc.month == "Mar") & (df_sa_psrc.year == 2000)].tail() + df_sa_psrc[(df_sa_psrc.month == "Apr") & (df_sa_psrc.year == 2000)].tail())

id,year,month,King & Snohomish: Total NonFarm,King & Snohomish: Consruction,King & Snohomish: FIRE,King & Snohomish: Manufacturing,King & Snohomish: Retail,King & Snohomish: Service,King & Snohomish: WTU,King & Snohomish: Government,...,Pierce: Education,Kitsap: Total NonFarm,Kitsap: Consruction,Kitsap: FIRE,Kitsap: Manufacturing,Kitsap: Retail,Kitsap: Service,Kitsap: WTU,Kitsap: Government,Kitsap: Education
122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
df_sa_psrc.tail()

id,year,month,King & Snohomish: Total NonFarm,King & Snohomish: Consruction,King & Snohomish: FIRE,King & Snohomish: Manufacturing,King & Snohomish: Retail,King & Snohomish: Service,King & Snohomish: WTU,King & Snohomish: Government,...,Pierce: Education,Kitsap: Total NonFarm,Kitsap: Consruction,Kitsap: FIRE,Kitsap: Manufacturing,Kitsap: Retail,Kitsap: Service,Kitsap: WTU,Kitsap: Government,Kitsap: Education
311,2015,Dec,1614000,89400,82900,170100,169700,759800,127200,110600,...,22700,88800,4200,0,2400,10600,37100,2600,31900,0
312,2016,Jan,1619100,89800,84300,168900,173500,761900,127700,110400,...,22600,88700,4100,0,2500,10700,37100,2500,31800,0
313,2016,Feb,1627000,91700,84000,167800,174300,766700,127500,110900,...,22700,88700,4000,0,2500,11000,36800,2600,31800,0
314,2016,Mar,1629200,92500,84300,167600,173800,765400,129300,112000,...,22600,88800,4200,0,2500,11000,36700,2600,31800,0
315,2016,Apr,1634100,91900,84700,167600,174900,769400,129100,112300,...,22500,88600,4200,0,2500,11000,36600,2600,31700,0


In [24]:
df_sa_psrc.query('month== "Mar"').tail()

id,year,month,King & Snohomish: Total NonFarm,King & Snohomish: Consruction,King & Snohomish: FIRE,King & Snohomish: Manufacturing,King & Snohomish: Retail,King & Snohomish: Service,King & Snohomish: WTU,King & Snohomish: Government,...,Kitsap: Education,Region: Construction,Region: FIRE,Region: Manufacturing,Region: Retail,Region: Service,Region: WTU,Region: Education,Region: Government,Region: Total NonFarm
266,2012,Mar,1446800,64200,77500,164700,143000,680600,114200,107700,...,0,84300,90000,183200,185300,831500,140100,116800,171800,1803000
278,2013,Mar,1490000,71100,80000,170500,150300,698800,115300,107200,...,0,92300,92800,189600,192500,851000,142900,118500,171100,1850700
290,2014,Mar,1533400,76500,80500,169200,159300,722200,118800,107900,...,0,99600,94100,188900,202900,880500,146900,121100,170800,1904800
302,2015,Mar,1578600,86400,81200,171100,167400,738800,122700,108800,...,0,110800,94900,190600,213800,898300,151500,124600,174800,1959300
314,2016,Mar,1629200,92500,84300,167600,173800,765400,129300,112000,...,0,118300,98100,187100,223000,932100,158400,126900,179100,2023000


In [ ]:
def process_esd_sa():
    start = time.time()
    
    
    
    
    
    
    
    
    processtime = end - start
    print "Data download completed. Download Time: " + "{0: .3}".format(processtime) + " Sec."
    